In [ ]:
from lexsub.results_parser import parse_results, ttest_df, ttest_df
from lexsub.results_parser import compute_statistical_measures,  compute_statistical_measures_lr
from lexsub.results_parser import get_results_opensesame, prettify, prettify2

import os
import sys
from glob import glob
import pandas as pd
pd.set_option('max_colwidth', None)


### 1SentencePerAnnotation experiments

In [ ]:
seed_exps_dir = "nExPerSent_verbs_randAllExps"
seed_exp = [(f"{seed_exps_dir}/01ExPerSent_verbs_rand01")]


expanded_exps_dir='expanded_nExPerSent_verbs_randAllExps'

expanded_exps1 = [
'01ExPerSent_verbs_rand01_expanded_lu',
'01ExPerSent_verbs_rand01_expanded_nouns-10pc',
'01ExPerSent_verbs_rand01_expanded_nouns-30pc',
'01ExPerSent_verbs_rand01_expanded_nouns-50pc',
]

expanded_exps2 = [
'01ExPerSent_verbs_rand01_expanded_roles',
'01ExPerSent_verbs_rand01_expanded_lu_roles_nouns-10pc',
'01ExPerSent_verbs_rand01_expanded_lu_roles_nouns-30pc',
'01ExPerSent_verbs_rand01_expanded_lu_roles_nouns-50pc',
]

pipeline1 = 'lugold_rolegold_nltk_nolemma_N2'
pipeline2 = 'lugold_rolegold_nltk_nolemma_role_stopwords_N2'


preds_model = 'xlnet_embs_hypers'
exps11 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps12 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps11.extend(exps12)

preds_model = 'bert'
exps21 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps22 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps21.extend(exps22)


exps = seed_exp
exps.extend(exps11)
exps.extend(exps21)


all_exps = exps
print(len(all_exps))


In [ ]:
output_model_dir = '../parser_workdir/step_logs'

# df = get_results_opensesame(all_exps, 
#                  output_model_dir,
#                  task_name="targetid")

# df = get_results_opensesame(all_exps, 
#                  output_model_dir,
#                  task_name="frameid")

df = get_results_opensesame(all_exps, 
                 output_model_dir,
                 task_name="argid")

df = prettify(df)



columns = ['preds_model', 'pipeline', 'dataset', 'task', 'f1', 'pre', 'rec']
columns = ['preds_model', 'pipeline', 'dataset', 'task', 'f1']
# df = df[columns]
df

# check number of runs per dataset/preds_model

# group_columns = ['preds_model', 'pipeline', 'dataset', 'task']

# dfg=df.groupby(group_columns, as_index=False).agg(['count']).reset_index()

# dfg


In [ ]:
results_df = compute_statistical_measures(df)
results_df


## Visualize - Learning Curve over  F1-score

### List of paths to experiments

In [ ]:
seed_exps_dir = "nPc_verbs_randAllExps"
seed_exps = """010pc_verbs
020pc_verbs
030pc_verbs
040pc_verbs
050pc_verbs
100pc_verbs""".split('\n')
seed_exps = [f"{seed_exps_dir}/{exp}" for exp in seed_exps]


expanded_exps_dir = 'expanded_nPc_verbs_randAllExps'

expanded_exps1 = """010pc_verbs_expanded_nouns-50pc
020pc_verbs_expanded_nouns-50pc
030pc_verbs_expanded_nouns-50pc
040pc_verbs_expanded_nouns-50pc
050pc_verbs_expanded_nouns-50pc
100pc_verbs_expanded_nouns-50pc""".split('\n')

expanded_exps2 = """010pc_verbs_expanded_lu_roles_nouns-50pc
020pc_verbs_expanded_lu_roles_nouns-50pc
030pc_verbs_expanded_lu_roles_nouns-50pc
040pc_verbs_expanded_lu_roles_nouns-50pc
050pc_verbs_expanded_lu_roles_nouns-50pc
100pc_verbs_expanded_lu_roles_nouns-50pc""".split('\n')

pipeline1 = 'lugold_rolegold_nltk_nolemma_N2'
pipeline2 = 'lugold_rolegold_nltk_nolemma_role_stopwords_N2'

preds_model = 'xlnet_embs_hypers'
exps11 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps12 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps11.extend(exps12)

preds_model = 'bert'
exps21 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline1}' for exp in expanded_exps1]
exps22 = [f'{expanded_exps_dir}/{exp}/{preds_model}/{pipeline2}' for exp in expanded_exps2]

exps21.extend(exps22)

exps = seed_exps
exps.extend(exps11)
exps.extend(exps21)

# ------------------------------------------------------------------------------------------
all_exps = exps

print(len(all_exps))


In [ ]:
output_model_dir = '../parser_workdir/step_logs'
df = get_results_opensesame(all_exps, output_model_dir)



df = prettify2(df)
# df
# columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
# group_columns = ['preds_model', 'pipeline', 'dataset', 'task', 'sample_size']

columns = ['preds_model', 'dataset', 'task', 'sample_size', 'f1', 'pre', 'rec']
group_columns = ['preds_model', 'dataset', 'task', 'sample_size']

df = df[columns]
df
# check number of runs per dataset/preds_model
# dfg=df.groupby(group_columns, as_index=False).agg(['count']).reset_index()

# dfg

In [ ]:
results_df = compute_statistical_measures_lr(df)
results_df


In [ ]:
# ----------------------------
def results_maps(dfg, 
                 base_dataset = 'nPercentData',
                 base_model="base",
                 augmented_datasets = ['augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc'],
                 augmented_models = ['bert', 'xlnet_embs_hypers'],
                 eval_measure="f1"):

    mean_scores = {}
    std_scores = {}
    p_values = {}
#     base model scores
    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['mean'].tolist()
    mean_scores[base_model] = {base_dataset:scores}

    scores = dfg.loc[(dfg['dataset'] == base_dataset) & (dfg['preds_model'] == base_model)][eval_measure]['std'].tolist()
    std_scores[base_model] = {base_dataset:scores}

# augmented model scores
    for m in augmented_models:

        all_models_means = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['mean'].tolist()
            all_models_means[ds] = scores

        all_models_std = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)][eval_measure]['std'].tolist()
            all_models_std[ds] = scores

        all_models_ps = {}
        for ds in augmented_datasets:
            scores = dfg.loc[(dfg['dataset'] == ds) & (dfg['preds_model'] == m)]['p_value'].tolist()
            all_models_ps[ds] = scores
            
        mean_scores[m] = all_models_means
        std_scores[m] = all_models_std
        p_values[m] = all_models_ps
        
    return mean_scores, std_scores, p_values



In [ ]:
mean_scores, std_scores, p_values = results_maps(results_df)
mean_scores, p_values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
%matplotlib inline

# ----------------------------------------------------------------------
N = [10,20,30,40,50, 100]

base_model = 'base'
augmented_models = ['bert', 'xlnet_embs_hypers']

datasets = ['nPercentData', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']
labels = ['seed', 'augmented-nouns-50pc', 'augmented-lexical unit-roles-nouns-50pc']

title = "Learning Curves"
subtitles = {'bert': 'BERT',
            'xlnet_embs_hypers': 'XLNet+embs'}

colors = ['k','c', 'y', 'r', 'brown']
# ----------------------------------------------------------------------
fig, axes = plt.subplots(1, 2, 
                         dpi=480,
                         figsize=(14, 4))

for j, m in enumerate(augmented_models):
    print(j, m)

    axes[j].set_title(f'Substitution model: {subtitles[m]}')
    axes[j].set_xlabel('Percentage of training data sampled randomly (dataset: Verbs)')
    axes[0].set_ylabel('F1-score (with gold frames)')
    axes[j].set_xticks(N)

    axes[j].set_yticks(range(26, 62, 2))
    axes[j].grid()
         
    i = -1
    for ds, label in zip(datasets, labels):
        i += 1
        print(ds, label)
        if label == 'seed': model = base_model
        else:
            model = m   
            ps = p_values[model][ds]
            # ------------------------------- to place P_values on optimally
            exp_names = list(mean_scores[model].keys())
            max_mean = [max(a,b) for a,b in zip(np.array(mean_scores[model][exp_names[0]]), np.array(mean_scores[model][exp_names[-1]]))]
            max_std = [max(a,b) for a,b in zip(np.array(std_scores[model][exp_names[0]]), np.array(std_scores[model][exp_names[-1]]))]
#             pY = [x+y for x,y in zip(max_mean, max_std)]
            pY = max_mean
        #     -------------------------------
            

        mean = np.array(mean_scores[model][ds])
        std = np.array(std_scores[model][ds])
        
        axes[j].fill_between(N, mean - std,
                             mean + std, alpha=0.1,
                             color=colors[i])

        axes[j].semilogx(N, mean, 'o-', color=colors[i],
                     label=labels[i], base=2, nonpositive='clip')
        pX = N
        p = -1
        if i==1:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 7
                    }
            for xx,yy in zip(pX,pY):
                p += 1
                #   place it below the lines
                if xx in [40, 100]:
                    axes[j].text(xx,yy-5, ps[p], fontdict=font)        
                #  place it above the lines
                else:
                    axes[j].text(xx,yy+3, ps[p], fontdict=font) 
        if i==2:
            font = {
#                 'family': 'serif',
                    'color':  colors[i],
                    'weight': 'bold',
                    'size': 7
                    }
            for xx,yy in zip(pX,pY):
                p += 1
                #   place it below the lines
                if xx in [40, 100]:
                    axes[j].text(xx,yy-6.5, ps[p], fontdict=font)        
                #  place it above the lines
                else:
                    axes[j].text(xx,yy+4.5, ps[p], fontdict=font) 
   

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes[:1]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
plt.figlegend( lines, labels, loc = 'lower center', ncol=3, labelspacing=0. , bbox_to_anchor=(0.5, -0.1))
# axes.legend()
plt.show()